# Data Analysis Project: E-Commerce Public Dataset
- **Name:** Muhammad Akbar Hamid
- **Email:** muhakbarhamid21@gmail.com
- **ID Dicoding:** muhakbarhamid21

## Defining Business Questions

1. **Which product categories have the highest and lowest sales volumes?**  
   This question aims to identify the top-performing product categories, as well as those that are underperforming in terms of sales volume, to guide inventory and marketing strategies.

2. **What are the monthly sales trends, and how have they evolved over time?**  
   This question focuses on analyzing the overall performance of monthly sales, allowing for the identification of seasonal patterns or other fluctuations over time.

3. **RFM Analysis**:
   - **When did customers last make a transaction, and what does this tell us about customer recency?**  
     This will help determine how recently customers have engaged with the business and identify those who may require re-engagement.

   - **How frequently are customers making purchases within recent months?**  
     This explores customer purchase frequency, allowing for the identification of repeat buyers and the overall engagement level of the customer base.

   - **Which are the top 5 customers in terms of monetary value spent in recent months?**  
     This question highlights the most valuable customers based on their total spending, aiding in loyalty programs or targeted marketing efforts.

4. **Which states have the highest and lowest total sales, and what regional patterns can be identified?**  
   This geographic analysis will reveal which regions are contributing the most to sales revenue and which areas may require more focus.

5. **What is the relationship between product pricing, shipping costs, and customer review scores?**  
   Through clustering analysis, this question seeks to understand how product price and shipping cost impact customer satisfaction, as reflected in review scores, to optimize pricing strategies and improve customer experience.

## Import All Packages/Libraries Used

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Data Wrangling

### Gathering Data

In [38]:
customers_df = pd.read_csv('data/customers_dataset.csv')
geolocation_df = pd.read_csv('data/geolocation_dataset.csv')
order_items_df = pd.read_csv('data/order_items_dataset.csv')
order_payments_df = pd.read_csv('data/order_payments_dataset.csv')
order_reviews_df = pd.read_csv('data/order_reviews_dataset.csv')
orders_df = pd.read_csv('data/orders_dataset.csv')
product_category_df =pd.read_csv('data/product_category_name_translation.csv')
products_df = pd.read_csv('data/products_dataset.csv')
sellers_df = pd.read_csv('data/sellers_dataset.csv')

### Assessing Data

#### Assess `customer` Dataset

In [ ]:
customers_df.info()

In [ ]:
print('Number of duplicates: ', customers_df.duplicated().sum())

In [ ]:
customers_df.describe()

#### Assess `geolocation` Dataset

In [ ]:
geolocation_df.info()

In [ ]:
duplicated_geo = geolocation_df.duplicated().sum()
print('Number of duplicates: ', duplicated_geo)

In [ ]:
geolocation_df.describe()

#### Assess `order_items` Dataset

In [ ]:
order_items_df.info()

In [46]:
order_items_df = order_items_df.drop('shipping_limit_date', axis=1)

In [ ]:
print('Number of duplicates: ', order_items_df.duplicated().sum())

In [ ]:
order_items_df.describe()

#### Assess `order_payments` Dataset

In [ ]:
order_payments_df.info()

In [ ]:
print('Number of duplicates: ', order_payments_df.duplicated().sum())

In [ ]:
order_payments_df.describe()

#### Assess `order_reviews` Dataset

In [ ]:
order_reviews_df.info()

In [53]:
order_reviews_df = order_reviews_df.drop(['review_creation_date','review_answer_timestamp', 'review_comment_title', 'review_comment_message'], axis=1)

In [ ]:
order_reviews_df.isna().sum()

In [ ]:
print('Number of duplicates:', order_reviews_df.duplicated().sum())

In [ ]:
order_reviews_df.describe()

#### Assess `orders` Dataset

In [ ]:
orders_df.info()

In [ ]:
orders_df.order_status.unique()

In [59]:
orders_df = orders_df.loc[orders_df['order_status'] != 'unavailable']
orders_df = orders_df.loc[orders_df['order_status'] != 'canceled']

In [ ]:
orders_df.isna().sum()

In [ ]:
print('Number of duplicates: ', orders_df.duplicated().sum())

In [ ]:
orders_df.describe()

#### Assess `product_category_name_translation` Dataset

In [ ]:
product_category_df.info()

In [ ]:
print('Number of duplicates: ', product_category_df.duplicated().sum())

#### Assess `product` Dataset

In [ ]:
products_df.info()

In [66]:
products_df = products_df[['product_id','product_category_name']]

In [ ]:
products_df.isna().sum()

In [ ]:
print('Number of duplicates: ', products_df.duplicated().sum())

In [ ]:
products_df.describe()

#### Assess `sellers` Dataset

In [ ]:
sellers_df.info()

In [ ]:
print('Number of duplicates: ', sellers_df.duplicated().sum())

### Cleaning Data

#### Remove Duplicates

The dataset that has duplicate data is only found in the `geolocation` dataset.

In [72]:
geolocation_df.drop_duplicates(inplace=True)

In [ ]:
print('Current Duplicate Count: ', geolocation_df.duplicated().sum())

#### Change Data Type

The dataset contains incorrect data types in the following columns:
- `orders_df`: The columns `order_purchase_timestamp`, `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, and `order_estimated_delivery_date` are currently not in the correct format. These columns should be converted to the datetime data type for proper handling of time-based operations and analysis.

In [74]:
date_time = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

for column in date_time:
  orders_df[column] = pd.to_datetime(orders_df[column])

In [ ]:
orders_df.info()

It can be observed that the columns `order_purchase_timestamp`, `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, and `order_estimated_delivery_date` now have the correct data type, which is **datetime**.

#### View Inaccuracy Data in **datetime**

The order of events is as follows:
`order_purchase_timestamp` < `order_approved_at` < `order_delivered_carrier_date` < `order_delivered_customer_date`, and `order_delivered_carrier_date` < `order_estimated_delivery_date`.

In [ ]:
wrong_order = orders_df[
    (orders_df['order_purchase_timestamp'] > orders_df['order_approved_at']) |
    (orders_df['order_approved_at'] > orders_df['order_delivered_carrier_date']) |
    (orders_df['order_delivered_carrier_date'] > orders_df['order_delivered_customer_date'])|
    (orders_df['order_delivered_carrier_date'] > orders_df['order_estimated_delivery_date'])
]
wrong_order

There are numerous incorrect data entries. As a solution, the incorrect values will be replaced with the previous datetime values from the respective order.

For example, if there is an error in the `order_approved_at` column, the value will be replaced with the data from `order_purchase_timestamp`.

In [77]:
orders_df.loc[orders_df['order_purchase_timestamp'] > orders_df['order_approved_at'], 'order_approved_at'] = orders_df['order_purchase_timestamp']
orders_df.loc[orders_df['order_approved_at'] > orders_df['order_delivered_carrier_date'], 'order_delivered_carrier_date'] = orders_df['order_approved_at']
orders_df.loc[orders_df['order_delivered_carrier_date'] > orders_df['order_delivered_customer_date'], 'order_delivered_customer_date'] = orders_df['order_delivered_carrier_date']
orders_df.loc[orders_df['order_delivered_carrier_date'] > orders_df['order_estimated_delivery_date'], 'order_estimated_delivery_date'] = orders_df['order_delivered_carrier_date']

In [ ]:
wrong_order = orders_df[
    (orders_df['order_purchase_timestamp'] > orders_df['order_approved_at']) |
    (orders_df['order_approved_at'] > orders_df['order_delivered_carrier_date']) |
    (orders_df['order_delivered_carrier_date'] > orders_df['order_delivered_customer_date'])|
    (orders_df['order_delivered_carrier_date'] > orders_df['order_estimated_delivery_date'])
]
wrong_order

The incorrect data has been resolved and is no longer present.

#### Handling Missing Value

Several datasets contain missing values:

- `orders` dataset: Missing values are found in the columns `order_approved_at`, `order_delivered_carrier_date`, and `order_delivered_customer_date`.
- `products` dataset: The column `product_category_name_id` has missing values.

These missing values should be addressed to ensure data quality and accuracy in the analysis.


In [ ]:
# dataset `order`
orders_df.isna().sum()

The missing values will be filled using the values from the preceding column.

In [80]:
orders_df.loc[orders_df['order_approved_at'].isna(), 'order_approved_at'] = orders_df['order_purchase_timestamp']
orders_df.loc[orders_df['order_delivered_carrier_date'].isna(), 'order_delivered_carrier_date'] = orders_df['order_approved_at']
orders_df.loc[orders_df['order_delivered_customer_date'].isna(), 'order_delivered_customer_date'] = orders_df['order_delivered_carrier_date']

In [ ]:
orders_df.isna().sum()

The data is now clean, with no missing values remaining.

In [ ]:
# dataset `product`
products_df.isna().sum()

Based on this dataset, the missing values cannot be filled (since category names must be accurate). Therefore, all rows containing missing values will be removed.

In [83]:
products_df = products_df.dropna()

In [ ]:
products_df.isna().sum()

The data is now clean, with no missing values remaining.

#### Fix Inaccuracy Value or Outliers

##### Dataset `order_items`

1. Outlier detection will be performed on the `price` and `freight_value` columns.
2. A merge between `products_df` and `product_category_df` will be done to retrieve the product names in English, resulting in `products_in_english_df`.
3. A merge between `order_items_df` and `products_in_english_df` will be conducted to determine whether the data points are truly outliers, resulting in `order_items_product_df`.

In [ ]:
order_items_df.nlargest(5, 'price')

It can be observed that many data points have high prices. Let's now check for the lowest prices.

In [ ]:
order_items_df.nsmallest(5, 'price')

It cannot yet be confirmed whether the data are outliers, as there may be genuinely expensive products. A merge with the product information will be performed to verify this.

In [87]:
products_in_english_df = pd.merge(
    left=products_df,
    right=product_category_df,
    how='left',
    left_on='product_category_name',
    right_on='product_category_name'
)

In [ ]:
order_items_product_df = pd.merge(
    left=order_items_df,
    right=products_in_english_df,
    how='left',
    left_on='product_id',
    right_on='product_id'
)
descending_order_products = order_items_product_df.sort_values(by='price', ascending=False)
descending_order_products.head()

In [ ]:
ascending_order_products = order_items_product_df.sort_values(by='price', ascending=True)
ascending_order_products.head(10)

It can be observed that there are no outliers in the `order_items` data for both `price` and `freight_value`. The high `freight_value` may be due to long shipping distances. As for the `order_item_id` column, since there are many product items, there are naturally multiple `order_item_id` values.

##### Dataset `order_payment`

1. To check for outliers in the `payment_value`, `payment_sequential`, and `payment_installments` columns, a merge will be performed on `order_items_product_df`, resulting in `order_payments_items_products_df`.
2. The outliers are mainly caused by duplicate `order_id values`, so those with the same `order_id` need to be consolidated, resulting in `result_order_payments_items_product_df`.

In [ ]:
order_payments_df.nlargest(5, 'payment_value')

In [91]:
order_payments_items_products_df = pd.merge(
    left=order_items_product_df,
    right=order_payments_df,
    how='left',
    left_on='order_id',
    right_on='order_id'
)

In [ ]:
descending_order_payments = order_payments_items_products_df.sort_values(by='payment_value', ascending=False)
descending_order_payments.head(10)

Jika dilihat dari order_id, kemungkinan dengan payment_value sebesar itu dikarenakan membeli dengan jumlah yang banyak.

In [ ]:
order_payments_items_products_df.describe(include='all')

Dapat dilihat bahwa adanya perbedaan antara banyaknya order_id dengan value yang unique pada order_id. Maka, kita harus:
1. Mengganti order_item_id dengan order_item dan mengambil nilai maksimalnya.
2. Menyatukan semua order_id yang sama

In [94]:
result_order_payments_items_products_df = order_payments_items_products_df.loc[order_payments_items_products_df.groupby('order_id')['order_item_id'].idxmax()]

In [ ]:
result_order_payments_items_products_df.sort_values(by='payment_value', ascending=False).head(10)

**Kesimpulan:**
Problem masalah terselesaikan. Payment_value yang tinggi disebabkan karena order_id yang menumpuk.

In [ ]:
# Berdasarkan Payment sequential
result_order_payments_items_products_df['payment_sequential'].unique()

Ternyata terdapat missing value. Hal ini dapat disebabkan karena penggabungan dataset (merge). Caranya adalah, kita akan menghapus semua missing value

In [ ]:
result_order_payments_items_products_df.isna().sum()

In [ ]:
result_order_payments_items_products_df.dropna(inplace=True)
result_order_payments_items_products_df.isna().sum()

Sudah tidak terdapat missing value.

In [ ]:
result_order_payments_items_products_df.sort_values(by='payment_sequential', ascending=False)

**Kesimpulan:**

Payment_sequential yang tinggi kemungkinan karena proses pembayaran atau urutan pembayarannya yang terbilang cukup banyak. Dan untuk payment_installment (pembayaran cicilan) tergantung dari kesepakatan awal pembelian. Jadi tidak mempengaruhi data outlier atau inaccuracy data jika < 100.

## Exploratory Data Analysis (EDA)

### Exploration of all the `orders` Dataset with the `products` Dataset

#### Merging The `orders` Dataset

In [100]:
orders_item_df = pd.merge(
    left=orders_df,
    right=order_items_df,
    how='left',
    left_on='order_id',
    right_on='order_id'
)
orders_payment_df = pd.merge(
    left=orders_item_df,
    right=order_payments_df,
    how='left',
    left_on='order_id',
    right_on='order_id'
)

all_orders_df = pd.merge(
    left=orders_payment_df,
    right=order_reviews_df,
    how='left',
    left_on='order_id',
    right_on='order_id'
)

In [ ]:
all_orders_df.info()

Due to the merging of datasets, a significant number of missing values may arise. It is essential to handle and clean these missing values to ensure data quality and accuracy for further analysis.

In [ ]:
all_orders_df.isna().sum()

In [103]:
all_orders_df.dropna(inplace=True)

In [ ]:
all_orders_df.isna().sum()

In [105]:
all_orders_df = all_orders_df.loc[all_orders_df.groupby('order_id')['order_item_id'].idxmax()]

Then, since we have already replaced the `order_item_id` with the largest `order_item_id`, rename it to `sum_order`.

In [ ]:
all_orders_df.rename(columns={'order_item_id': 'qty_order'}, inplace=True)
all_orders_df.head(5)

#### Merging The `products` Dataset

In [107]:
products_df = pd.merge(
    left=products_df,
    right=product_category_df,
    how='left',
    left_on='product_category_name',
    right_on='product_category_name'
)

In [ ]:
products_df.isna().sum()

In [109]:
products_df.dropna(inplace=True)

#### Merging The `orders` Dataset and The `products` Dataset

In [110]:
orders_products_df = pd.merge(
    left=all_orders_df,
    right=products_df,
    how='left',
    left_on='product_id',
    right_on='product_id'
)

In [ ]:
orders_products_df.isna().sum()

Missing values have occurred due to the merging of datasets. These missing values will be removed to ensure data integrity for further analysis.

In [112]:
orders_products_df.dropna(inplace=True)

In [ ]:
orders_products_df.isna().sum()

#### Identifying Which Products Have The Highest and Lowest Review Scores.

In [ ]:
orders_products_df.groupby(by='product_category_name_english').agg({
    'product_id':'nunique',
    'qty_order':'sum',
    'review_score':'mean'
}).sort_values(by=['review_score','qty_order'], ascending=False)

It can be observed that:
- The highest review score is for the cds_dvds_musicals category, with a score of 4.6. This category has a total of 46 orders, but only consists of 1 product type.
- The lowest review score is for the security_and_services category, with a score of 5. This category has a total of 2 orders and includes 2 different product types.

This highlights a significant difference in both review scores and order volumes between the two categories.

#### Examining The Time Intervals

- The time interval between when the order was placed and when it was approved.
- The time interval from approval to handing over to the shipping service.
- The shipping duration.
- The time interval between the estimated delivery date and the actual delivery to the customer.
- The time interval from the order placement date to the actual delivery to the customer.

In [ ]:
orders_products_df.order_status.unique()

In [ ]:
orders_products_df.info()

In [117]:
# time interval between order placement and approval
orders_products_copy = orders_products_df.copy()
apply_time = orders_products_copy['order_approved_at'] - orders_products_copy['order_purchase_timestamp']
apply_time = apply_time.apply(lambda x: x.total_seconds())
orders_products_df['apply_time'] = round(apply_time/86400)

In [118]:
# time interval from approval to handover to the shipping service
shipped_time = orders_products_copy.loc[orders_products_copy['order_status'] != 'created']
shipped_time = shipped_time['order_delivered_carrier_date'] - shipped_time['order_approved_at']
shipped_time = shipped_time.apply(lambda x: x.total_seconds())
orders_products_df['shipped_time'] = round(shipped_time/86400)

In [119]:
# shipping time interval
customer_gets_order = orders_products_copy.loc[~orders_products_copy['order_status'].isin(['created', 'processing', 'approved', 'invoiced'])]
customer_gets_order = customer_gets_order['order_delivered_customer_date'] - customer_gets_order['order_delivered_carrier_date']
customer_gets_order = customer_gets_order.apply(lambda x: x.total_seconds())
orders_products_df['customer_gets_order'] = round(customer_gets_order/86400)

In [120]:
# time interval between the estimated delivery and actual delivery to the customer
estimated_range = orders_products_copy.loc[~orders_products_copy['order_status'].isin(['created', 'processing', 'approved', 'invoiced'])]
estimed_range = estimated_range['order_estimated_delivery_date'] - estimated_range['order_delivered_carrier_date']
estimed_range = estimed_range.apply(lambda x: x.total_seconds())
orders_products_df['estimated_range'] = round(estimed_range/86400)

In [121]:
# time interval from the order date to the delivery to the customer
range_order = orders_products_copy['order_delivered_customer_date'] - orders_products_copy['order_purchase_timestamp']
range_order = range_order.apply(lambda x: x.total_seconds())
orders_products_df['range_order'] = round(range_order/86400)

In [ ]:
orders_products_df.agg({
    'apply_time':['mean', 'min', 'max'],
    'shipped_time':['mean', 'min', 'max'],
    'customer_gets_order':['mean', 'min', 'max'],
    'estimated_range':['mean','min','max'],
    'range_order':['mean', 'min', 'max']
})

**Conclusion:**
- The time interval between order placement and approval ranges from 0 to 60 days, with an average of less than 1 day (0.23 days).
- The time interval from approval to handover to the shipping service ranges from 0 to 107 days, with an average of 2-3 days (7.883 days).
- The shipping time interval ranges from 0 to 205 days, with an average of 9 days (9.14 days).
- The time interval between the estimated delivery and when the order reaches the customer ranges from 0 to 148 days, with an average estimated delivery time of 20-21 days (20.42 days).
- The time interval from the order date to the delivery to the customer ranges from 0 to 208 days (0 likely due to orders being canceled or input errors that have been corrected), with an average of 12 days (12.30 days).

#### Analyzing The Rating Based on The Number of Orders

In [ ]:
orders_products_df.order_status.unique()

In [ ]:
orders_products_df.groupby(by='review_score').agg({
    'qty_order': 'sum',
    'product_category_name_english': 'unique',
    'payment_value': ['min', 'max']
}).sort_values(('qty_order', 'sum'), ascending=False)

It can be observed that there are 62,284 orders with a rating of 5, 20,887 orders with a rating of 4, and 13,438 orders with a rating of 1. This indicates a need for evaluation and improvement in products under the product_category_name_english that received a rating of 1. Addressing these issues is crucial to prevent further customer dissatisfaction and to avoid lowering the overall rating of these products in the future.

### Merging The `customers` Dataset

Renaming the columns customer_zip_code_prefix and geolocation_zip_code_prefix to zip_code_prefix.

In [125]:
geolocation_df.rename(columns={'geolocation_zip_code_prefix': 'zip_code_prefix'}, inplace=True)
customers_df.rename(columns={'customer_zip_code_prefix': 'zip_code_prefix'}, inplace=True)

In [126]:
customers_df = pd.merge(
    left=customers_df,
    right=geolocation_df,
    how='left',
    left_on='zip_code_prefix',
    right_on='zip_code_prefix'
)

In [ ]:
customers_df.info()

In [ ]:
customers_df.isna().sum()

In [ ]:
customers_df.dropna()

In [ ]:
customers_df.describe(include='all')

It can be observed that the customer_id and customer_unique_id should have the same values for both count and unique entries. This means there are duplicate customer_id entries. We will retain only one customer_id for each unique value.

In [131]:
drop_duplicates = customers_df.drop_duplicates('customer_id', keep='first')
customers_df = pd.concat([ 
    drop_duplicates, customers_df[~customers_df.duplicated('customer_id', keep=False)]
])
customers_df = customers_df.reset_index(drop=True)

In [132]:
customers_df = customers_df.drop_duplicates(subset='customer_id', keep='first').reset_index(drop=True)

In [ ]:
customers_df.describe(include='all')

### Merging The `sellers` Dataset

Renaming the seller_zip_code_prefix column, then merging it.

In [134]:
sellers_df.rename(columns={'seller_zip_code_prefix': 'zip_code_prefix'}, inplace=True)

In [135]:
sellers_df = pd.merge(
    left=sellers_df,
    right=geolocation_df,
    how='left',
    left_on='zip_code_prefix',
    right_on='zip_code_prefix'
)

In [ ]:
sellers_df.isna().sum()

In [137]:
sellers_df.dropna(inplace=True)

The same issue occurs in the sellers dataset, where there are non-unique seller IDs. Let's check it.

In [ ]:
sellers_df.describe(include='all')

The same seller IDs will be merged.

In [139]:
drop_duplicates = sellers_df.drop_duplicates('seller_id', keep='first')
sellers_df = pd.concat([ 
    drop_duplicates, sellers_df[~sellers_df.duplicated('seller_id', keep=False)]
])
sellers_df = sellers_df.reset_index(drop=True)

In [140]:
sellers_df = sellers_df.drop_duplicates(subset='seller_id', keep='first').reset_index(drop=True)

In [ ]:
sellers_df.describe(include='all')

The data has been corrected. Next, the task is to merge all the datasets.

### Explore All Datasets

#### Merging All Datasets

In [ ]:
orders_products_customers_df = pd.merge(
    left=orders_products_df,
    right=customers_df,
    how='left',
    left_on='customer_id',
    right_on='customer_id'
)

all_df = pd.merge(
    left=orders_products_customers_df,
    right=sellers_df,
    how='left',
    left_on='seller_id',
    right_on='seller_id'
)

all_df.info()

In [143]:
all_df.dropna(inplace=True)

#### View Monthly Sales

In [144]:
all_df.loc[:, 'month_year'] = all_df['order_purchase_timestamp'].dt.strftime('%Y-%m')
monthly_salaries = all_df.groupby('month_year')['payment_value'].sum().reset_index()

In [ ]:
monthly_salaries.sort_values(by='payment_value', ascending=False)

It can be observed that the highest revenue occurred in April 2018, amounting to $1,115,553.30, while the lowest revenue was in December 2016, totaling $19.62.

#### Top 5 Best Selling and Least Selling Product Categories

In [ ]:
product_category = all_df.groupby(by='product_category_name_english').agg({
    'product_id': 'nunique',
    'qty_order': 'sum',
    'payment_value': ['min', 'max']
})
product_category.reset_index()

In [ ]:
most_purchases = product_category.sort_values(by=('qty_order', 'sum'), ascending=False)
most_purchases.head(5)

It can be seen that the top 5 best-selling product categories are bed_bath_table, health_beauty, sports_leisure, furniture_decor, and computers_accessories. These categories dominate in terms of total orders.

In [ ]:
least_purchases = product_category.sort_values(by=('qty_order', 'sum'), ascending=True)
least_purchases.head(5)

On the other hand, the categories with the fewest sales are security_and_services, fashion_childrens_clothes, la_cuisine, cds_dvds_musicals, and arts_and_craftmanship. These categories have significantly lower order volumes compared to the top sellers.

## Visualization & Explanatory Analysis

### Question 1: Which product categories have the highest and lowest sales volumes?

In [ ]:
products_visualization = all_df.groupby('product_category_name_english')['qty_order'].sum().reset_index()
bottom = products_visualization.sort_values(by='qty_order', ascending=True).head(5)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,8))
colors = ["#72BCD4", "#D3D3D3", "#D3D3D3", "#D3D3D3", "#D3D3D3"]

# Top 5 product categories by quantity ordered
sns.barplot(x='qty_order', y='product_category_name_english', 
            data=products_visualization.sort_values(by='qty_order', ascending=False).head(5), 
            hue='product_category_name_english', palette=colors, ax=ax[0], dodge=False, width=0.5)

ax[0].set_xlabel('Total Quantity Ordered', fontsize=14, fontweight='bold')
ax[0].set_ylabel('Product Category Name', fontsize=14, fontweight='bold')
ax[0].set_title('Top 5 Best Selling Product Categories', loc='center', fontsize=16, fontweight='bold')
ax[0].tick_params(axis='y', labelsize=12)
ax[0].tick_params(axis='x', labelsize=12)
ax[0].legend(title='Product Category', loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, title_fontsize=14)

# Bottom 5 product categories by quantity ordered
sns.barplot(x='qty_order', y='product_category_name_english', 
            data=bottom.head(5), 
            hue='product_category_name_english', palette=colors, ax=ax[1], dodge=False, width=0.5)

ax[1].set_xlabel('Total Quantity Ordered', fontsize=14, fontweight='bold')
ax[1].set_ylabel('Product Category Name', fontsize=14, fontweight='bold')
ax[1].set_title('Top 5 Worst Selling Product Categories', loc='center', fontsize=16, fontweight='bold')
ax[1].tick_params(axis='y', labelsize=12)
ax[1].tick_params(axis='x', labelsize=12)
ax[1].legend(title='Product Category', loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12, title_fontsize=14)

plt.suptitle('Top 5 Best and Worst Selling Product Categories by Quantity Ordered', fontsize=20, fontweight='bold')
plt.tight_layout(rect=[0, 0, 0.85, 0.95])
plt.show()


**Top 5 Best Selling Product Categories:**
- The bed_bath_table category leads as the top-selling product, with a significantly higher quantity sold compared to other categories.
- The other top categories—health_beauty, sports_leisure, furniture_decor, and computers_accessories—also demonstrate strong sales, but they lag behind bed_bath_table in total quantity ordered.

**Top 5 Worst Selling Product Categories:**
- The security_and_services category has the lowest quantity of products sold, followed by fashion_childrens_clothes, la_cuisine, cds_dvds_musicals, and arts_and_craftmanship, which also show very low sales volumes.
- The gap between the top and bottom categories is vast, showing that certain product categories perform much better in terms of sales compared to others, with bottom categories barely registering significant sales.

### Question 2: What are the monthly sales trends, and how have they evolved over time?

In [ ]:
monthly_sales = all_df.groupby('month_year')['payment_value'].sum().reset_index()

fig = plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_sales, x="month_year", y="payment_value", marker="o", linestyle="-", color='#4C72B0')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel("Total Payment Value ($)", fontsize=14, fontweight='bold')
plt.xlabel("Month-Year", fontsize=14, fontweight='bold')
plt.title("Monthly Sales Volume Over Time", fontsize=16, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Based on the chart, the following conclusions can be drawn:
1. The timeline in the chart starts from September 2016 and ends in September 2018.
2. There are noticeable increases in sales in the months of October 2016, January 2017, February 2017, March 2017, May 2017, July 2017, August 2017, September 2017, October 2017, November 2017, January 2018, March 2018, April 2018, and July 2018.
3. There are decreases in sales during December 2016, April 2017, June 2017, December 2017, February 2018, May 2018, June 2018, August 2018, and September 2018.
4. A significant spike occurred in November 2017, along with a consistent upward trend from January 2017 to March 2017.
5. A drastic drop in sales is seen in September 2018, which could be due to incomplete data processing or ongoing record updates for that month.

This indicates general growth in sales over time, with fluctuations possibly related to seasonality or data completion issues for September 2018.

### Question 3: The RFM (Recency, Frequency, Monetary) analysis



In [ ]:
now = all_df['order_purchase_timestamp'].max()
rfm_df = all_df.groupby(by='customer_id', as_index=False).agg({
    'order_purchase_timestamp':lambda x: (now - x.max()).days,
    'order_id': 'count',
    'payment_value':'sum'
})
rfm_df.columns = ['customer_id', 'recency', 'frequency', 'monetary']
rfm_df.head()

Because the customer IDs are quite long, we will replace them with numeric IDs. This change does not have any impact, as we are only interested in seeing when the last transaction occurred, how frequently the customer makes purchases, and the total revenue generated by each customer.

In [ ]:
rfm_df['numeric_id'] = pd.factorize(rfm_df['customer_id'])[0] + 1
rfm_df.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30,6))
colors = sns.color_palette("husl", 5)

# Bar plot for Recency (Last Purchase)
sns.barplot(y="recency", x='numeric_id', data=rfm_df.sort_values(by='recency', ascending=True).head(5), hue='numeric_id', palette=colors, ax=ax[0], dodge=False, width=0.5)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].set_title('Last Purchase (days)', loc='center', fontsize=18, fontweight='bold')
ax[0].tick_params(axis='x', labelsize=15)
ax[0].tick_params(axis='y', labelsize=12)
ax[0].legend(title='Customer ID', fontsize=12, title_fontsize=14, loc='upper right', bbox_to_anchor=(1.15, 1))

# Bar plot for Frequency (Number of Purchases)
sns.barplot(y='frequency', x='numeric_id', data=rfm_df.sort_values(by='frequency', ascending=False).head(5), hue='numeric_id', palette=colors, ax=ax[1], dodge=False, width=0.5)
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].set_title('Purchase Frequency', loc='center', fontsize=18, fontweight='bold')
ax[1].tick_params(axis='x', labelsize=15)
ax[1].tick_params(axis='y', labelsize=12)
ax[1].legend(title='Customer ID', fontsize=12, title_fontsize=14, loc='upper right', bbox_to_anchor=(1.15, 1))

# Bar plot for Monetary (Total Money Spent)
sns.barplot(y='monetary', x='numeric_id', data=rfm_df.sort_values(by='monetary', ascending=False).head(5), hue='numeric_id', palette=colors, ax=ax[2], dodge=False, width=0.5)
ax[2].set_ylabel(None)
ax[2].set_xlabel(None)
ax[2].set_title('Total Amount Spent', loc='center', fontsize=18, fontweight='bold')
ax[2].tick_params(axis='x', labelsize=15)
ax[2].tick_params(axis='y', labelsize=12)
ax[2].legend(title='Customer ID', fontsize=12, title_fontsize=14, loc='upper right', bbox_to_anchor=(1.15, 1))

plt.suptitle('Best Customers Based on RFM Parameters', fontsize=22, fontweight='bold')
plt.tight_layout()
plt.show()


Based on the data visualization above, the following conclusions can be drawn:
1. **Recency (Last Purchase)**: Some customers have made transactions within the last 0 to 4 days, with the most recent transaction made by the customer with ID `28005`.
2. **Frequency (Purchase Frequency)**: Most customers have made purchases once in the last few months, indicating relatively infrequent purchase patterns.
3. **Monetary (Total Amount Spent)**: The amount of money spent by customers varies. The customer with ID `8201` has spent the most, showing significant financial involvement compared to other customers.

From the three RFM parameters, the top customers who are the most active and spend the most can be identified within the analyzed period.

### Question 4: Which states have the highest and lowest total sales, and what regional patterns can be identified?

In [ ]:
sales_by_state = all_df.groupby('customer_state')['payment_value'].sum().sort_values(ascending=True)
plt.figure(figsize=(12, 6))
sales_by_state.plot(kind='barh', color='skyblue', edgecolor='black')
plt.title('Total Sales by State', fontsize=16, fontweight='bold')
plt.ylabel('State', fontsize=14, fontweight='bold')
plt.xlabel('Total Sales ($)', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6, axis='x')
plt.tight_layout()
plt.show()


Based on the total sales by state chart, it can be concluded that the state with the highest sales is São Paulo (SP), with a significant gap compared to other states. On the other hand, the states with the lowest sales are Roraima (RR), Amapá (AP), Acre (AC), and Amazonas (AM). This indicates an uneven distribution of sales, where larger or more developed states tend to have much higher sales volumes.

### Question 5: What is the relationship between product pricing, shipping costs, and customer review scores?

In [ ]:
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12,8))
scatter = plt.scatter(all_df['price'], all_df['freight_value'], 
                      c=all_df['review_score'], cmap='viridis', s=80, alpha=0.7, edgecolor='w', linewidth=0.5)
cbar = plt.colorbar(scatter)
cbar.set_label('Review Score', fontsize=12)
plt.xlabel('Product Price ($)', fontsize=14)
plt.ylabel('Shipping Cost ($)', fontsize=14)
plt.title('Relationship between Product Price, Shipping Cost, and Review Score', fontsize=16, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


Most products are in the lower range of price and shipping cost. However, higher-priced products tend to receive better review scores, even though their shipping costs are also higher. This suggests that other factors, such as quality, influence customers' purchasing decisions.

## Conclusion

1. **Which product categories have the highest and lowest sales volumes?**  
   The top-selling product categories are **bed_bath_table, health_beauty, sports_leisure, furniture_decor,** and **computers_accessories**. Conversely, the categories with the least sales are **security_and_services, fashion_childrens_clothes, la_cuisine, cds_dvds_musicals,** and **art_and_craftmanship**. This indicates that home-related and technology products are the most popular, while niche categories like arts and children's fashion have lower demand.

2. **What are the monthly sales trends, and how have they evolved over time?**  
   The analysis of monthly sales trends reveals key patterns:
   - The data spans from September 2016 to September 2018.
   - Significant sales increases occurred in months like October 2016, January to March 2017, May, July, and September 2017, as well as November 2017 and early 2018.
   - Declines were observed in December 2016, April, June, and December 2017, and sporadically in 2018.
   - November 2017 showed a substantial spike in sales, possibly driven by promotional events, followed by a consistent growth period in early 2017.
   - The sharp decline in September 2018 may be attributed to incomplete data collection at the time of analysis.

3. **RFM Analysis**:
   - **Recency**: Most customers made purchases within the last 0-4 days of the dataset, with the most recent transaction by customer ID 28005.
   - **Frequency**: The majority of customers have made only **one purchase** in recent months, indicating a large proportion of single-purchase behavior.
   - **Monetary**: Spending varies widely, but customer ID 8201 is the highest spender, showing a significant contribution to overall revenue.

4. **Which states have the highest and lowest total sales, and what regional patterns can be identified?**  
   From the geo-analysis, **Sao Paulo** stands out as the state with the highest sales by a significant margin, indicating a strong customer base in this region. On the other hand, **Roraima (RR), Amapa (AP), Acre (AC),** and **Amazonas (AM)** are the states with the lowest sales, suggesting potential opportunities for market expansion or targeted campaigns in these regions.

5. **What is the relationship between product pricing, shipping costs, and customer review scores?**  
   The clustering analysis shows that most products are within the lower price and shipping cost range. However, higher-priced products tend to receive better review scores, even though their shipping costs are also relatively high. This suggests that customers are willing to pay a premium for higher quality products, indicating that price alone is not a significant barrier when the perceived value and customer satisfaction are high.